<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch01_CH02_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning
https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0

In [1]:
import os
import math

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

In [5]:
def get_model(input_shape):
  inputs = keras.Input(input_shape)
  base_model = EfficientNetB0(
      input_shape = input_shape,
      weights = 'imagenet',
      include_top = False,   # True : 분류까지 다 가져옴, False : 피처벡터 추출까지만 가져옴
      pooling = 'avg',
  )

  x = base_model(inputs)
  outputs = layers.Dense(1, activation= 'sigmoid')(x)
  model = keras.Model(inputs, outputs)
  return model

input_shape =(256, 256, 3)
model = get_model(input_shape)

adam = keras.optimizers.Adam(learning_rate = 0.0001)

model.compile(
    optimizer = adam,
    loss = 'binary_crossentropy',
    metrics = 'accuracy'
)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, 1280)              4049571   
 )                                                               
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 4050852 (15.45 MB)
Trainable params: 4008829 (15.29 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [6]:
import albumentations as A
import cv2

class Augmentation:
  def __init__(self, size, mode ='train'):
    if mode =='train':
      # Declare an augmentation pipeline
      self.transform = A.Compose([
          A.HorizontalFlip(p=0.5), # 좌우반전, p(적용할 확률)
          A.ShiftScaleRotate(
              p = 0.5,
              shift_limit = 0.05,
              scale_limit = 0.05,
              rotate_limit = 15,
          ), # 이동%, 확대축소%, 회전각도 제약걸기
          A.CoarseDropout(
              p = 0.5,
              max_holes =8,
              max_height = int(0.1 * size),
              max_width = int(0.1 * size),
          ),
          A.RandomBrightnessContrast(p=0.2),
      ])

  def __call__(self, **kwargs):
    if self.transform:
      aumented = self.transform(**kwargs)
      img = aumented['image']
      return img


In [7]:
class DataGenerator(keras.utils.Sequence):
    def __init__(
        self,
        batch_size,
        csv_path,
        fold,
        image_size,
        mode ='train',
        shuffle = True):

        self.batch_size = batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle

        self.df = pd.read_csv(csv_path)

        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode =='val':
            self.df = self.df[self.df['fold'] == self.fold]


        ### Remove invalid files
        ### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['filename'].isin(invalid_filenames)]
        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()


    def __len__(self):
        return math.ceil(len(self.df)/self.batch_size)

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1)* self.batch_size
        data = self.df.iloc[strt:fin]

        batch_x, batch_y = self.get_data(data)

        return np.array(batch_x), np.array(batch_y)

    def get_data(self, data):
        batch_x =[]
        batch_y =[]

        for _, r in data.iterrows():
            file_name = r['filename']
            print(file_name)

            image = cv2.imread(f'/content/MyDrive/data/images/{file_name}.jpg')
            print(f'/content/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # input image resize (배치로 묶기위해)
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':
              image = image.astype('uint8')
              image = self.transform(image = image)

            # rescaling
            image = image /255.

            # 고양이 1 강아지 2 -> 고양이 0 강아지 1
            label = int(r['species']) -1

            batch_x.append(image)
            batch_y.append(label)
        return batch_x, batch_y



    def on_epoch_end(self): # callback function
        if self.shuffle:
            self.df = self.df.sample(frac =1).reset_index(drop=True)


In [ ]:
csv_path = '/content/drive/MyDrive/data/kfolds.csv'
train_generator = DataGenerator(
    batch_size = 128,
    csv_path =csv_path,
    fold = 1,
    image_size = 256,
    mode ='train',
    shuffle = True
)

val_generator = DataGenerator(
    batch_size = 128,
    csv_path =csv_path,
    fold = 1,
    image_size = 256,
    mode ='val',
    shuffle = True
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 10,
    verbose = 1
)